# Makine Öğrenimini Kullanarak Otistik Spektrum Bozukluğu Taraması

Nörogelişim bozukluklarının erken teşhisi tedaviyi iyileştirebilir ve ilgili sağlık maliyetlerini önemli ölçüde azaltabilir. Bu projede, davranışsal özelliklere ve bireysel özelliklere dayalı Otistik Spektrum Bozukluğunu (ASD) teşhis etmek için denetimli öğrenmeyi kullanacağız. Daha spesifik olarak, Keras'ı kullanarak bir sinir ağı kuracağız ve dağıtacağız.

Bu proje, 292 hasta için tarama verilerini içeren UCI Makine Öğrenim Deposu tarafından sağlanan bir veri kümesini kullanacaktır. Veri kümesi aşağıdaki URL'de bulunabilir:
https://archive.ics.uci.edu/ml/datasets/Autistic+Spectrum+Disorder+Screening+Data+for+Children++


Hemen başlayalım! İlk olarak, bu projede kullanacağımız birkaç kütüphaneyi içe aktaracağız.

In [101]:
import sys
import pandas as pd
import sklearn
import keras

print ('Python: {}'.format(sys.version))
print ('Pandas: {}'.format(pd.__version__))
print ('Sklearn: {}'.format(sklearn.__version__))
print ('Keras: {}'.format(keras.__version__))

Python: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
Pandas: 1.0.1
Sklearn: 0.22.1
Keras: 2.3.1


## 1.Verileri Yükleme


Verileri UCI Machine Learning Depository'den alacağız; ancak, veriler bir csv veya txt dosyasında bulunmadığından, sıkıştırılmış zip dosyasını indirip verileri manuel olarak çıkarmamız gerekir. Bu tamamlandıktan sonra, bilgileri Pandalar kullanarak bir metin dosyasından okuyacağız.

Dönüştürme için: https://pulipulichen.github.io/HTML5-BayesNet-Tools/arff2csv/ adresini kullanabilirsiniz.

In [102]:
# Veri kümesini Yükleme
file = 'csv_result-Autism-Child-Data.csv'

# Csv Dosyasını Okuma
data = pd.read_table(file, sep = ',', index_col = None)

In [103]:
# Veri kümesinin yapısı yazdırma, bu sayede verimiz hakkında bilgi edinebilriiz
print ('Shape of DataFrame: {}'.format(data.shape))
print (data.loc[0])
print(data.info())

Shape of DataFrame: (292, 22)
id                          1
A1_Score                    1
A2_Score                    1
A3_Score                    0
A4_Score                    0
A5_Score                    1
A6_Score                    1
A7_Score                    0
A8_Score                    1
A9_Score                    0
A10_Score                   0
age                         6
gender                      m
ethnicity              Others
jundice                    no
austim                     no
contry_of_res          Jordan
used_app_before            no
result                      5
age_desc           4-11 years
relation               Parent
Class/ASD                  NO
Name: 0, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               292 non-null    int64 
 1   A1_Score         292 non-null    int64 
 2

In [104]:
# Veri kümesi hakkında istatistiksel bilgileri yazdırma
data.describe()

,id,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,result
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,146.500000,0.633562,0.534247,0.743151,0.551370,0.743151,0.712329,0.606164,0.496575,0.493151,0.726027,6.239726
std,84.437354,0.482658,0.499682,0.437646,0.498208,0.437646,0.453454,0.489438,0.500847,0.500811,0.446761,2.284882
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,73.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,146.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,6.000000
75%,219.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000
max,292.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000


## 2. Veri önişlemesi

Bu veri kümesi için birden fazla önişleme adımı gerekecektir. İlk olarak, veri kümemizde (attributes) sinir ağımızı eğitirken kullanmak istemediğimiz sütunlar var. Önce bu sütunları bırakacağız. İkinci olarak, verilerimizin çoğu dizi(string) kullanılarak rapor edilir; sonuç olarak, verilerimizi kategorik etiketlere dönüştüreceğiz. Önişlemimiz sırasında, veri kümesini X ve Y veri kümelerine böleriz; burada X, tahmin için kullanmak istediğimiz tüm özelliklere ve Y sınıf etiketlerine sahiptir.

In [105]:
# İstenmeyen dataları çıkarma
data = data.drop(['result', 'age_desc'], axis=1)

In [106]:
# Modeli eğitmek için verileri gruplara ayırma
x = data.drop(['Class/ASD'], 1)
y = data['Class/ASD']

In [107]:
# Verileri kategorik değerlere çevirme (one-hot-encoded vectors)
X = pd.get_dummies(x)
Y = pd.get_dummies(y)

In [108]:
# Oluşturduğumuz kategorik sütunları inceleyelim
print (X.columns.values)
print(Y.columns.values)

['id' 'A1_Score' 'A2_Score' 'A3_Score' 'A4_Score' 'A5_Score' 'A6_Score'
 'A7_Score' 'A8_Score' 'A9_Score' 'A10_Score' 'age_10' 'age_11' 'age_4'
 'age_5' 'age_6' 'age_7' 'age_8' 'age_9' 'age_?' 'gender_f' 'gender_m'
 'ethnicity_?' 'ethnicity_Asian' 'ethnicity_Black' 'ethnicity_Hispanic'
 'ethnicity_Latino' 'ethnicity_Middle Eastern ' 'ethnicity_Others'
 'ethnicity_Pasifika' 'ethnicity_South Asian' 'ethnicity_Turkish'
 'ethnicity_White-European' 'jundice_no' 'jundice_yes' 'austim_no'
 'austim_yes' 'contry_of_res_Afghanistan' 'contry_of_res_Argentina'
 'contry_of_res_Armenia' 'contry_of_res_Australia' 'contry_of_res_Austria'
 'contry_of_res_Bahrain' 'contry_of_res_Bangladesh' 'contry_of_res_Bhutan'
 'contry_of_res_Brazil' 'contry_of_res_Bulgaria' 'contry_of_res_Canada'
 'contry_of_res_China' 'contry_of_res_Costa Rica' 'contry_of_res_Egypt'
 'contry_of_res_Europe' 'contry_of_res_Georgia' 'contry_of_res_Germany'
 'contry_of_res_Ghana' 'contry_of_res_India' 'contry_of_res_Iraq'
 'contry_of_r

## 3. Veri Kümesini Eğitim ve Test Veri Kümelerine ayırın

Sinir ağımızı eğitmeye başlamadan önce, veri kümesini eğitim ve test veri kümelerine ayırmamız gerekir. Bu, eğitim bittikten sonra yeni verilere ne kadar genelleştirileceğini belirlemek için ağımızı test etmemizi sağlayacaktır. Scikit-learn tarafından sağlanan train_test_split () işlevini kullanırken bu adım inanılmaz derecede kolay!

In [109]:
from sklearn import model_selection

# Oluşturduğumuz veri gruplarını(X,Y) eğitim ve test aşaması için bölme
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size = 0.2)

In [110]:
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print (Y_test.shape)

(233, 97)
(59, 97)
(233, 2)
(59, 2)


## 4. Sinir ağı kurma - Keras

Bu projede, ağımızı kurmak ve eğitmek için Keras'ı kullanacağız. Bu model nispeten basit olacak ve sadece yoğun (Dense) katmanları kullanacaktır. Bu en yaygın sinir ağı katmanıdır. Ağın bir gizli katmanı olacak, bir Adam optimize edici ve kategorik bir çapraz-girişim kaybı(categorical crossentropy loss) olacak. Öğrenme hızı, her katmandaki nöron sayısı veya bu projedeki aktivasyon fonksiyonları gibi parametreleri optimize etmekle zaman kaybetmeyeceğiz; Bununla birlikte, zamanınız varsa, bu parametreleri manuel olarak ayarlamak ve sonuçları gözlemlemek, işlevleri hakkında bilgi edinmek için harika bir yoldur!

In [111]:
#Sinir ağını kurmak - Keras

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Keras modelini kurmak için fonksiyon oluşturma

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=97, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    
    # Model Derlemesi(compile)
    adam = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model = create_model()

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 8)                 784       
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 10        
Total params: 830
Trainable params: 830
Non-trainable params: 0
_________________________________________________________________
None


## 5. Sinir ağını eğitme

Şimdi eğlence zamanı :)
Bir Keras modelini eğitmek, model.fit () işlevini çağırmak kadar basittir.

In [112]:
# Modeli eğitim veri kümelerimizle eğitme
model.fit(X_train, Y_train, epochs=50, batch_size=10, verbose = 1)

Epoch 1/50
233/233 [==============================] - 0s 509us/step - loss: 0.7012 - accuracy: 0.4635
Epoch 2/50
233/233 [==============================] - 0s 94us/step - loss: 0.6929 - accuracy: 0.5279
Epoch 3/50
233/233 [==============================] - 0s 94us/step - loss: 0.6921 - accuracy: 0.5279
Epoch 4/50
233/233 [==============================] - 0s 98us/step - loss: 0.6896 - accuracy: 0.5279
Epoch 5/50
233/233 [==============================] - 0s 98us/step - loss: 0.6886 - accuracy: 0.5279
Epoch 6/50
233/233 [==============================] - 0s 98us/step - loss: 0.6859 - accuracy: 0.5451
Epoch 7/50
233/233 [==============================] - 0s 103us/step - loss: 0.6823 - accuracy: 0.5622
Epoch 8/50
233/233 [==============================] - 0s 103us/step - loss: 0.6776 - accuracy: 0.5923
Epoch 9/50
233/233 [==============================] - 0s 98us/step - loss: 0.6718 - accuracy: 0.7382
Epoch 10/50
233/233 [==============================] - 0s 111us/step - loss: 0.6610 - ac

## 6. Modeli Test Etme ve Performans Metrikleri

Modelimiz eğitildiğine göre, performansını test veri kümesinde test etmemiz gerekiyor. Model bu bilgiyi daha önce hiç görmedi; sonuç olarak, test veri seti, modelin eğitim aşamasında kullanılmayan bilgilere genelleme yapıp yapamayacağını belirlememizi sağlar. Bu amaçla scikit-learn tarafından sağlanan metriklerden bazılarını kullanacağız!

In [116]:
# kategorik model için tahminler kullanarak sınıflandırma raporu oluşturabilir
from sklearn.metrics import classification_report, accuracy_score

predictions = model.predict_classes(X_test)
predictions

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1], dtype=int64)

In [117]:
print('Results for Categorical Model')
print(accuracy_score(Y_test[['YES']], predictions))
print(classification_report(Y_test[['YES']], predictions))

Results for Categorical Model
0.864406779661017
              precision    recall  f1-score   support

           0       0.83      0.89      0.86        28
           1       0.90      0.84      0.87        31

    accuracy                           0.86        59
   macro avg       0.86      0.87      0.86        59
weighted avg       0.87      0.86      0.86        59

